<a href="https://colab.research.google.com/github/faizanahemad/eva/blob/master/assignment-14/DN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Resources**

- XLA

    - [Google Notebook example](https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/compiler/xla/g3doc/tutorials/xla_compile.ipynb)

    - [Blog](https://medium.com/@xianbao.qian/use-xla-with-keras-3ca5d0309c26)
    - [Example Enabling XLA](https://github.com/tensorflow/models/blob/7212436440eaa11293ca84befcc5d8327109ea76/official/utils/misc/keras_utils.py#L158)
    
- Mixed Precision

- Augmentation Libs
    - [imgaug](https://github.com/aleju/imgaug)
    - [albumentations](https://github.com/albu/albumentations)
    - [Automold](https://github.com/UjjwalSaxena/Automold--Road-Augmentation-Library)
    - [Tensorflow Examples](https://www.wouterbulten.nl/blog/tech/data-augmentation-using-tensorflow-data-dataset/)
    - [PIL ImageOps Examples](https://hhsprings.bitbucket.io/docs/programming/examples/python/PIL/ImageOps.html)
    
- Tensorflow References
    - [Using Numpy functions](https://www.tensorflow.org/api_docs/python/tf/numpy_function)
    - [Using Python functions](https://www.tensorflow.org/api_docs/python/tf/py_function)

# Imports

In [1]:
import os
import gc


BATCH_SIZE = 512 #@param {type:"integer"}
MOMENTUM = 0.9 #@param {type:"number"}
LEARNING_RATE = 0.5 #@param {type:"number"}
WEIGHT_DECAY = 1e-3 #@param {type:"number"}
EPOCHS = 13 #@param {type:"integer"}


jobs = int(os.cpu_count()/2)

float_rep = "float16"


In [2]:
import tensorflow as tf

# tf.keras.backend.set_epsilon(1e-4)
# tf.keras.backend.set_floatx('float16')

tf.enable_eager_execution()
# tf.random.set_random_seed(1234)


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ec2-user/a

In [4]:
import numpy as np
import skimage
print(np.__version__)
print(skimage.__version__)


import time, math
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.contrib.eager as tfe
import gc
print(tf.__version__)
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
% matplotlib inline
%config InlineBackend.figure_format='retina'

import copy
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from tensorflow.core.protobuf import rewriter_config_pb2
import os
from importlib import reload
import lib
reload(lib)
from lib import *
from data_science_utils import misc

import pandas as pd


import imgaug.augmenters as iaa
from matplotlib import cm
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,RandomFog,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, ChannelDropout, ChannelShuffle,RandomContrast
)

W0821 17:47:47.895162 139782799902528 deprecation_wrapper.py:119] From /home/ec2-user/SageMaker/eva/assignment-14/lib.py:206: The name tf.keras.backend.set_session is deprecated. Please use tf.compat.v1.keras.backend.set_session instead.

W0821 17:47:47.895967 139782799902528 deprecation_wrapper.py:119] From /home/ec2-user/SageMaker/eva/assignment-14/lib.py:208: The name tf.keras.backend.get_session is deprecated. Please use tf.compat.v1.keras.backend.get_session instead.



1.17.0
0.15.0
1.14.0


<module 'lib' from '/home/ec2-user/SageMaker/eva/assignment-14/lib.py'>

In [5]:
logger.info('This is a warning')


I0821 17:47:49.074721 139782799902528 <ipython-input-5-bf20c9c65a9e>:1] This is a warning


# Persist/Read Train Set

## Persist

In [53]:
import dill as pickle
with open('train_data_s2.pkl', 'wb') as file:
    pickle.dump(train_data, file)

## Read 

In [29]:
import dill as pickle
with open('train_data_s2.pkl', 'rb') as file:
    train_data = pickle.load(file)

# GridSearch

In [25]:
def read_and_append_to_results(filename, result_object=None):
    from pathlib import Path
    import ast
    my_file = Path(filename)
    if my_file.is_file():
        results = misc.load_list_per_line(filename)
    else:
        results = []
    
    if result_object is not None:
        results.append(str(result_object))
        misc.save_list_per_line(results, filename)
    
    def lit_eval(r):
        try:
            return ast.literal_eval(r)
        except:
            return None
    return list(filter(lambda x: x is not None,map(lit_eval,results)))

In [26]:
def run_model(model,opt,global_step,BATCH_SIZE,EPOCHS,train_data,x_test, y_test,len_train,log_test_acc=False):
    len_test = len(x_test)
    train_accs = []
    test_accs = []
    test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE).prefetch(len_test)


    

    logger.info("Starting Training")
    t = time.time()
    for epoch in range(EPOCHS):
        tf.keras.backend.set_learning_phase(1)    
        train_loss = test_loss = train_acc = test_acc = 0.0
        train_set = train_data[epoch]
        for (x, y) in train_set:
            with tf.GradientTape() as tape:
                loss, correct = model(x, y)

            var = model.trainable_variables
            grads = tape.gradient(loss, var)
            for g, v in zip(grads, var):
                g += v * (WEIGHT_DECAY/(epoch+1)) * BATCH_SIZE
            opt.apply_gradients(zip(grads, var), global_step=global_step)

            train_loss += loss.numpy()
            train_acc += correct.numpy()
            train_accs.append(train_acc / len_train)
        if log_test_acc or epoch==EPOCHS-1:    
            tf.keras.backend.set_learning_phase(0)
            for (x, y) in test_set:
                loss, correct = model(x, y)
                test_loss += loss.numpy()
                test_acc += correct.numpy()
            test_accs.append(test_acc / len_test)
            logger.debug(msg("epoch = %2s"%epoch,'||train=> loss: %.3f' %(train_loss / len_train), 'acc: %.3f' % (train_acc / len_train), '||val=> loss: %.3f' % (test_loss / len_test), 'val acc: %.3f' %(test_acc / len_test), '%.1fs'%(time.time() - t)))
        logger.debug(msg("trained for epoch = ",epoch,"train acc = ",train_accs[-1]))
            
    time_spent = time.time() - t
    logger.info(msg("Train acc = ",train_accs[-1],"Test acc =",test_accs[-1],"Time Taken = ",time_spent))
    return train_accs[-1],test_accs[-1],time_spent
    
    


In [27]:
def build_augmenters(da_config,train_mean):
    cutout_config = da_config["cutout_config"]
    cifar10_augs_config = da_config["cifar10_augs_config"]
    hue_config = da_config["hue_config"]
    cutout_mapper = get_numpy_wrapper(get_cutout_eraser(**cutout_config),"cutout")
    cifar10_mapper = get_numpy_wrapper(CIFAR10Policy(**cifar10_augs_config, fillcolor=tuple(train_mean.astype(int)), log=False),"AutoAug")
    hue_mapper = get_hue_aug(hue_config["max_delta"])
    full_wrapper = get_multimapper([hflip_mapper,cutout_mapper,cifar10_mapper,hue_mapper])
    logger.debug("Augmentation Functions Built")
    return full_wrapper


def process_full_augmentation_all_epochs(full_wrapper, EPOCHS, normalize, x_train, y_train):
    train_data = {}
    len_train = len(x_train)
    normalize = get_numpy_wrapper(normalize, Tout=tf.float32)
    t = time.time()
    for epoch in range(EPOCHS):
        train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(full_wrapper, num_parallel_calls=tf.data.experimental.AUTOTUNE).map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE).shuffle(len_train).batch(len_train).prefetch(len_train)
        i = 0
        for x,y in train_set:
            xt = x.numpy()
            logger.debug(msg("X Shape = ",xt.shape[0], "Train Length =",len_train))
            assert x.numpy().shape[0]==len_train
            train_set = (xt,y.numpy())
            i = i+1
        assert i==1
        train_data[epoch] = train_set
        logger.debug(msg("Augmentation Epoch = ",epoch,"Time Spent = %.1f" % (time.time() - t)))
        
    train_sets = {}
    for epoch in range(EPOCHS):
        key = (epoch)%(max(train_data.keys())+1)
        train_set = train_data[key]
        train_set =tf.data.Dataset.from_tensor_slices(train_set).batch(BATCH_SIZE).prefetch(len_train)
        train_sets[epoch] = train_set
    time_taken = time.time() - t
    logger.info(msg("Augmentation Done, Time Taken = %.1f"%(time_taken)))
    return train_sets,time_taken
    
    

def fetch_cifar_data():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    classes = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
    len_train, len_test = len(x_train), len(x_test)
    y_train = y_train.astype('int64').reshape(len_train)
    y_test = y_test.astype('int64').reshape(len_test)
    train_mean = np.mean(x_train, axis=(0,1,2))
    train_std = np.std(x_train, axis=(0,1,2))

    def normalize(x):
        return ((x.astype('float32') - train_mean) / train_std).astype('float32')

    
    pad4 = lambda x: np.pad(x, [(0, 0), (4, 4), (4, 4), (0, 0)], mode='reflect')
    x_train = pad4(x_train)
    x_test = normalize(x_test)
    logger.info("Data Fetching Done")
    return x_train, y_train, x_test, y_test, classes,len_train,len_test, normalize, train_mean


     

def model_builder(model_config,BATCH_SIZE, len_train):
    
    batches_per_epoch = len_train//BATCH_SIZE + 1
    model = FNet(**model_config["model"])
    
    enable_olr = model_config["optimizer"]["enable_olr"]
    enable_momentum = enable_olr
    max_lr = model_config["optimizer"]["max_lr"]
    max_momentum = model_config["optimizer"]["max_momentum"]
    global_step = tf.train.get_or_create_global_step()
    if enable_olr:
        mid_epoch = model_config["optimizer"]["mid_epoch"]
        start_lr = model_config["optimizer"]["start_lr"]
        
        end_lr = model_config["optimizer"]["end_lr"]
        pre_end_lr = model_config["optimizer"]["pre_end_lr"]
        pre_end_epoch = model_config["optimizer"]["pre_end_epoch"]
        
        enable_momentum = model_config["optimizer"]["enable_momentum"]
        if enable_momentum:
            min_momentum = model_config["optimizer"]["min_momentum"]
            momentum_schedule =  lambda t: np.interp([t], [0, mid_epoch, pre_end_epoch, EPOCHS], [max_momentum, min_momentum, max_momentum, max_momentum])[0] 
            momentum_func = lambda: momentum_schedule(global_step/batches_per_epoch)
    
        lr_schedule = lambda t: np.interp([t], [0, mid_epoch, pre_end_epoch, EPOCHS], [start_lr, max_lr, pre_end_lr, end_lr])[0] # LR = 0.75
        lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE
    
    
    
    opt = tf.train.MomentumOptimizer(learning_rate=lr_func if enable_olr else max_lr, momentum=momentum_func if enable_momentum else max_momentum, use_nesterov=True)
    logger.info("Model Built")
    return model,opt,global_step




def runner(configs):
    # fetch Data
    x_train, y_train, x_test, y_test, classes,len_train,len_test, normalize,train_mean = fetch_cifar_data()
    prev_da_config = None
    train_data = None
    results = []
    for i,config in enumerate(configs):
        reset_keras()
        logger.info("Testing for Config: %s",i)
        _ = gc.collect()
        model_config = config["model_config"]
        da_config = config["augmentation_config"]
        BATCH_SIZE = config["training_config"]["BATCH_SIZE"]
        EPOCHS = config["training_config"]["EPOCHS"]
        save_file = config["training_config"]["save_file"]
        
        same_as_previous_config = prev_da_config == da_config
        prev_da_config = da_config
        model,opt,global_step = model_builder(model_config,BATCH_SIZE, len_train)
        
        if not same_as_previous_config:
            augmenter = build_augmenters(da_config,train_mean)
            train_data,time_taken = process_full_augmentation_all_epochs(augmenter, EPOCHS, normalize, x_train, y_train)
            
        
        train_acc,test_acc,time_spent = run_model(model,opt,global_step,BATCH_SIZE,EPOCHS,train_data,x_test, y_test,len_train,log_test_acc=False)
        
        result = dict(config=config,results=dict(train=train_acc,test=test_acc,training_time=time_spent,augmentation_time=time_taken))
        read_and_append_to_results(save_file, result)
        results.append(result)
        del model
        model = None
    logger.info("Grid Search Complete, Total Results Count = %s",len(results))
    assert len(results)==len(configs)
    return results
    
        


In [28]:
augmentation_config = dict(cutout_config=dict(s_l=0.04, s_h=0.06,max_erasures_per_image=1),
                           hue_config=dict(max_delta=0.1),
                           cifar10_augs_config=dict(proba=0.5, enabled_policies=[("rotate",5, 15,),("shearX",0.05, 0.15,),("shearY",0.05, 0.15,)]))

model_config = dict(model=dict(start_kernels=64,sparse_bn=True,thin_block=False),
                    optimizer=dict(enable_olr=True,max_lr=0.5,
                                   start_lr=0.001,pre_end_lr=0.01,end_lr=0.005,
                                   mid_epoch=5,pre_end_epoch=13,
                                   max_momentum=0.9,enable_momentum=False))

training_config = dict(BATCH_SIZE=512,EPOCHS=15,save_file="results.txt")

test_config = dict(augmentation_config=augmentation_config,model_config=model_config,training_config=training_config)

test_config_2 = copy.deepcopy(test_config)
test_config_2["model_config"]["optimizer"]["enable_momentum"] = True
test_config_2["model_config"]["optimizer"]["max_momentum"] = 0.95
test_config_2["model_config"]["optimizer"]["min_momentum"] = 0.85

test_config_3 = copy.deepcopy(test_config)
test_config_3["model_config"]["optimizer"]["enable_momentum"] = True
test_config_3["model_config"]["optimizer"]["max_momentum"] = 0.95
test_config_3["model_config"]["optimizer"]["min_momentum"] = 0.85
test_config_3["model_config"]["optimizer"]["max_lr"] = 0.4


test_config_4 = copy.deepcopy(test_config)
test_config_4["model_config"]["optimizer"]["enable_momentum"] = True
test_config_4["model_config"]["optimizer"]["max_momentum"] = 0.95
test_config_4["model_config"]["optimizer"]["min_momentum"] = 0.85
test_config_4["model_config"]["optimizer"]["pre_end_epoch"] = 12

test_config_5 = copy.deepcopy(test_config)
test_config_5["model_config"]["optimizer"]["enable_momentum"] = True
test_config_5["model_config"]["optimizer"]["max_momentum"] = 0.92
test_config_5["model_config"]["optimizer"]["min_momentum"] = 0.82

test_config_6 = copy.deepcopy(test_config)
test_config_6["model_config"]["optimizer"]["enable_momentum"] = True
test_config_6["model_config"]["optimizer"]["max_momentum"] = 0.98
test_config_6["model_config"]["optimizer"]["min_momentum"] = 0.88

results = runner([test_config,test_config_2,test_config_3,test_config_4,test_config_5,test_config_6])



I0821 18:43:25.656678 139782799902528 <ipython-input-27-e77edb9b4089>:59] Data Fetching Done
I0821 18:43:26.181678 139782799902528 <ipython-input-27-e77edb9b4089>:109] Testing for Config: 0
I0821 18:43:26.335893 139782799902528 <ipython-input-27-e77edb9b4089>:95] Model Built
I0821 18:52:09.650828 139782799902528 <ipython-input-27-e77edb9b4089>:38]  Augmentation Done, Time Taken = 523.3
I0821 18:52:10.229412 139782799902528 <ipython-input-26-198f9c213b46>:10] Starting Training
I0821 18:54:56.086319 139782799902528 <ipython-input-26-198f9c213b46>:40]  Train acc =  0.91854 Test acc = 0.9114 Time Taken =  165.85596656799316
I0821 18:54:56.241785 139782799902528 <ipython-input-27-e77edb9b4089>:109] Testing for Config: 1
I0821 18:54:56.381575 139782799902528 <ipython-input-27-e77edb9b4089>:95] Model Built
I0821 18:54:56.467541 139782799902528 <ipython-input-26-198f9c213b46>:10] Starting Training
I0821 18:57:45.651390 139782799902528 <ipython-input-26-198f9c213b46>:40]  Train acc =  0.92318 T

In [12]:
results

[{'config': {'augmentation_config': {'cutout_config': {'s_l': 0.04,
     's_h': 0.06,
     'max_erasures_per_image': 1},
    'hue_config': {'max_delta': 0.1},
    'cifar10_augs_config': {'proba': 0.5,
     'enabled_policies': [('rotate', 5, 15),
      ('shearX', 0.1, 0.2),
      ('shearY', 0.1, 0.2)]}},
   'model_config': {'model': {'start_kernels': 64,
     'sparse_bn': True,
     'thin_block': False},
    'optimizer': {'enable_olr': True,
     'max_lr': 0.5,
     'start_lr': 0.001,
     'pre_end_lr': 0.01,
     'end_lr': 0.005,
     'mid_epoch': 5,
     'pre_end_epoch': 13,
     'max_momentum': 0.95,
     'enable_momentum': True,
     'min_momentum': 0.85}},
   'training_config': {'BATCH_SIZE': 512,
    'EPOCHS': 15,
    'save_file': 'results.txt'}},
  'results': {'train': 0.85312,
   'test': 0.8749,
   'training_time': 174.3953332901001,
   'augmentation_time': 522.2028503417969}},
 {'config': {'augmentation_config': {'cutout_config': {'s_l': 0.04,
     's_h': 0.06,
     'max_erasur

In [96]:
results

[{'config': {'augmentation_config': {'cutout_config': {'s_l': 0.04,
     's_h': 0.06,
     'max_erasures_per_image': 1},
    'hue_config': {'max_delta': 0.1},
    'cifar10_augs_config': {'proba': 0.5,
     'enabled_policies': [('rotate', 5, 15),
      ('shearX', 0.1, 0.2),
      ('shearY', 0.1, 0.2)]}},
   'model_config': {'model': {'start_kernels': 64,
     'sparse_bn': True,
     'thin_block': True},
    'optimizer': {'enable_olr': True,
     'max_lr': 0.5,
     'start_lr': 0.001,
     'pre_end_lr': 0.01,
     'end_lr': 0.005,
     'mid_epoch': 5,
     'pre_end_epoch': 13,
     'max_momentum': 0.9,
     'enable_momentum': False}},
   'training_config': {'BATCH_SIZE': 512,
    'EPOCHS': 15,
    'save_file': 'results.txt'}},
  'results': {'train': 0.87154,
   'test': 0.8858,
   'training_time': 150.91348099708557,
   'augmentation_time': 432.3216440677643}}]